## Check if the training process is exactly the same

In [ ]:
import torch
import lightning as L
import rpad.pyg.nets.pointnet2 as pnp

from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from python_ml_project_template.datasets.flow_trajectory import FlowTrajectoryDataModule
from python_ml_project_template.datasets.flowbot import FlowBotDataModule

torch.set_printoptions(precision=15)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### 1: Original flowbot

In [ ]:
from python_ml_project_template.models.flow_predictor import FlowPredictorTrainingModule

def run_flowbot():
    with initialize(version_base='1.3', config_path="../configs/"):
        cfg = compose(config_name="train_flowbot")
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.set_float32_matmul_precision("medium")

    # Run 2 epoch of original flowbot model
    L.seed_everything(cfg.seed)
    flowbot_datamodule = FlowBotDataModule(
        root=cfg.dataset.data_dir,
        batch_size=cfg.training.batch_size,
        num_workers=cfg.resources.num_workers,
        n_proc=cfg.training.n_proc,  # Add n_proc
    )
    flowbot_train_loader = flowbot_datamodule.train_dataloader()
    flowbot_val_loader = flowbot_datamodule.val_dataloader()
    flowbot_unseen_loader = flowbot_datamodule.unseen_dataloader()

    # TODO: Training
    mask_channel = 1 if cfg.training.mask_input_channel else 0
    flowbot_network = pnp.PN2Dense(
        in_channels=mask_channel, out_channels=3, p=pnp.PN2DenseParams()
    )

    flowbot_model = FlowPredictorTrainingModule(
        flowbot_network, training_cfg=cfg.training
    )
    flowbot_trainer = L.Trainer(
        accelerator="gpu",
        devices=cfg.resources.gpus,
        precision="16-mixed",
        max_steps=10
    )

    flowbot_trainer.fit(
        flowbot_model,
        flowbot_train_loader,
        [flowbot_train_loader, flowbot_val_loader, flowbot_unseen_loader],
    )
    return flowbot_model


In [ ]:
flowbot_model = run_flowbot()

### 2: Trajectory=1

In [ ]:
from python_ml_project_template.models.flow_trajectory_predictor import FlowTrajectoryTrainingModule

def run_single_trajectory():
    with initialize(version_base='1.3', config_path="../configs/"):
        cfg = compose(config_name="train_trajectory")
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.set_float32_matmul_precision("medium")
    L.seed_everything(cfg.seed)
    # TODO: Dataset
    traj_datamodule = FlowTrajectoryDataModule(
        root=cfg.dataset.data_dir,
        batch_size=cfg.training.batch_size,
        num_workers=cfg.resources.num_workers,
        trajectory_len=cfg.training.trajectory_len,
        n_proc=cfg.training.n_proc,  # Add n_proc
        seed=cfg.seed,
    )
    traj_train_loader = traj_datamodule.train_dataloader()
    traj_val_loader = traj_datamodule.val_dataloader()
    traj_unseen_loader = traj_datamodule.unseen_dataloader()

    mask_channel = 1 if cfg.training.mask_input_channel else 0
    traj_network = pnp.PN2Dense(
        in_channels=mask_channel,
        out_channels=3 * 1,
        p=pnp.PN2DenseParams(),
    )

    traj_trainer = L.Trainer(
        accelerator="gpu",
        devices=cfg.resources.gpus,
        precision="16-mixed",
        max_steps=10,
        callbacks=[],
    )
    traj_model = FlowTrajectoryTrainingModule(traj_network, training_cfg=cfg.training)
    traj_trainer.fit(
        traj_model,
        traj_train_loader,
        [traj_train_loader, traj_val_loader, traj_unseen_loader],
    )
    return traj_model


In [ ]:
traj_model = run_single_trajectory()

### The parameters

In [ ]:
for p1, p2 in zip(flowbot_model.parameters(), traj_model.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print(p1.data - p2.data)